In [1]:
# use on colab to mount your drive
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#change directory to the folder in your drive on colab
%cd drive/MyDrive/TFRecords_4_videos/

/content/drive/MyDrive/TFRecords_4_videos


In [1]:
#%load_ext autoreload
#%autoreload 2

import pathlib 
import json
import imageio 
import numpy as np 
from tqdm import tqdm
import pandas as pd 

# Formatting Something-Something-v1

In [4]:
ss_files = pathlib.Path('.')
# how we access fileID (when the path in train.txt is not absolute)
datafile_prefix = pathlib.Path('dataset/data')

labels = pd.read_csv(ss_files/'labels.csv', sep='\n', header=None)
labels = list(labels[0])
train = pd.read_csv(ss_files/'training_list.csv', header=None)
val = pd.read_csv(ss_files/'val_list.csv',  header=None)
test = pd.read_csv(ss_files/'test_list.csv',  header=None)


train.columns = ['FileID', 'Labels']
val.columns = ['FileID', 'Labels']
test.columns = ['FileID', 'Labels']
print('Labels: ', labels[:])
test.head()


Labels:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]


,FileID,Labels
0,001_001_005.mp4,1
1,001_002_005.mp4,1
2,001_003_005.mp4,1
3,001_004_005.mp4,1
4,001_005_005.mp4,1


In [5]:
datafile_path = pathlib.Path('./')/'records/tfrecords'
datafile_path.mkdir(parents=True, exist_ok=True)

for df, name in zip([train, val,test], ['train.txt', 'val.txt','test.txt']):
    # compute the index of each label 
    #print(df['Labels'].values)
    df['label_idx'] = [labels.index(v) for v in df['Labels'].values]
    df['label_idx']+=1
    #print(df.label_idx.values)
    # write to train.txt or val.txt
    with open(str(datafile_path/name), 'w') as f: 
        for _, row in tqdm(df.iterrows()): 
            line = '{} {}\n'.format(row['FileID'], row['label_idx'])
            f.write(line)

1350it [00:00, 15879.18it/s]
450it [00:00, 13643.07it/s]
450it [00:00, 12156.78it/s]


In [ ]:
# lets look at an example line in the file 
with open(str(datafile_path/'train.txt'), 'r') as f: 
    lines = f.readlines()
print('Example in file (fileID, label): ', lines[10])

Example in file (fileID, label):  001_004_002.mp4 1



In [4]:

# to make sure the paths are correct -- lets access an example:
example_fileid = lines[np.random.randint(len(lines))].split(' ')[0]
# format using datafile_prefix
example_path = datafile_prefix/'{}'.format(example_fileid)
print(example_path)

NameError: name 'lines' is not defined

In [ ]:
# success! 
list(example_path.iterdir())[:5]

NotADirectoryError: [WinError 267] The directory name is invalid: 'dataset\\data\\047_003_001.mp4'

# Writing TFRecords

In [6]:
from tfrecords import TFRecords4Video
# we will save the files in the same place as our datafiles
tfrecords_save_path = datafile_path
# datafile_file and datafile_prefix were already created and checked 
tfrv = TFRecords4Video(tfrecords_save_path, datafile_path, datafile_prefix, 'video')



In [7]:
## run this line to create all the tf records for each split 
tfrv.create_tfrecords()

Starting processing split test.
Extracting paths and labels...


100%|██████████| 450/450 [00:00<00:00, 14049.07it/s]



Number of files not found: 0 / 450
Writing to TFRecords...
Splitting 450 examples into 1.00 GB shards
Using 8 processes...


100%|██████████| 90/90 [29:06<00:00, 19.41s/it]

Finished processing split test.


In [1]:
import tensorflow as tf

print(tf.test.gpu_device_name())

In [15]:
!ffmpeg --version

ffmpeg version 5.0-full_build-www.gyan.dev Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 11.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-shared --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable

# Loading TFRecords

In [ ]:
# get a str(path) to every training shard 
train_path = tfrecords_save_path/'train/'
train_shards = [str(path) for path in train_path.iterdir() if '.tfrecord' == path.suffix]
train_shards

['records\\tfrecords\\train\\0-shard0.tfrecord',
 'records\\tfrecords\\train\\1-shard0.tfrecord',
 'records\\tfrecords\\train\\10-shard0.tfrecord',
 'records\\tfrecords\\train\\100-shard0.tfrecord',
 'records\\tfrecords\\train\\101-shard0.tfrecord',
 'records\\tfrecords\\train\\102-shard0.tfrecord',
 'records\\tfrecords\\train\\103-shard0.tfrecord',
 'records\\tfrecords\\train\\104-shard0.tfrecord',
 'records\\tfrecords\\train\\105-shard0.tfrecord',
 'records\\tfrecords\\train\\106-shard0.tfrecord',
 'records\\tfrecords\\train\\107-shard0.tfrecord',
 'records\\tfrecords\\train\\108-shard0.tfrecord',
 'records\\tfrecords\\train\\109-shard0.tfrecord',
 'records\\tfrecords\\train\\11-shard0.tfrecord',
 'records\\tfrecords\\train\\110-shard0.tfrecord',
 'records\\tfrecords\\train\\111-shard0.tfrecord',
 'records\\tfrecords\\train\\112-shard0.tfrecord',
 'records\\tfrecords\\train\\113-shard0.tfrecord',
 'records\\tfrecords\\train\\114-shard0.tfrecord',
 'records\\tfrecords\\train\\115-shar

In [ ]:
#import tensorflow.compat.v1 as tf 
import tensorflow as tf 

from tfrecords import parse_example

# create dataset with train shards
# customize adding: .map(your_tf_function) after '.map(parse_example)\'
# Note: sharding allows us to shuffle the data efficiently
dataset = tf.data.TFRecordDataset(train_shards).map(parse_example).batch(1)
#    .shuffle(100)\


for batch in dataset:
  video_data, labels, filenames=batch
  print(video_data.shape, labels, filenames)


# Note: for batchsize > 1 you'll have to resize the images all to the same size 
# i.e .map(your_resize_function)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(1, 54, 576, 352, 3) tf.Tensor([0], shape=(1,), dtype=int64) tf.Tensor([b'dataset\\data\\001_001_001.mp4'], shape=(1,), dtype=string)
(1, 59, 576, 352, 3) tf.Tensor([0], shape=(1,), dtype=int64) tf.Tensor([b'dataset\\data\\001_001_002.mp4'], shape=(1,), dtype=string)
(1, 58, 576, 352, 3) tf.Tensor([0], shape=(1,), dtype=int64) tf.Tensor([b'dataset\\data\\001_001_003.mp4'], shape=(1,), dtype=string

KeyboardInterrupt: 

In [ ]:
!ffmpeg.exe -version

ffmpeg version 5.0-full_build-www.gyan.dev Copyright (c) 2000-2022 the FFmpeg developers
built with gcc 11.2.0 (Rev5, Built by MSYS2 project)
configuration: --enable-gpl --enable-version3 --enable-shared --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvd